In [0]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import *

In [0]:
import numpy as np

In [0]:
with open('/content/Bean.txt', 'r',encoding='utf-8') as f:
    x = f.readlines()

In [78]:
data=str(x)
type(data)

str

In [79]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 4573


In [80]:
# encode 4 words -> 1 word
sequences = list()
for i in range(4, len(encoded)):
	sequence = encoded[i-4:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 38561


In [81]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 5


In [0]:
# split into input and output elements
sequences = np.array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

In [83]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(64))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 4, 10)             45730     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                19200     
_________________________________________________________________
dense_8 (Dense)              (None, 4573)              297245    
Total params: 362,175
Trainable params: 362,175
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
model.fit(X, y, epochs=500, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
38561/38561 [==============================] - 19s 484us/step - loss: 6.0440 - accuracy: 0.1338
Epoch 2/500
38561/38561 [==============================] - 19s 483us/step - loss: 5.5003 - accuracy: 0.2029
Epoch 3/500
38561/38561 [==============================] - 18s 476us/step - loss: 5.2180 - accuracy: 0.2146
Epoch 4/500
38561/38561 [==============================] - 18s 476us/step - loss: 5.0591 - accuracy: 0.2297
Epoch 5/500
38561/38561 [==============================] - 18s 473us/step - loss: 4.9117 - accuracy: 0.2441
Epoch 6/500
38561/38561 [==============================] - 18s 472us/step - loss: 4.7723 - accuracy: 0.2522
Epoch 7/500
38561/38561 [==============================] - 18s 473us/step - loss: 4.6403 - accuracy: 0.2602
Epoch 8/500
38561/38561 [==============================] - 18s 473us/step - loss: 4.5145 - accuracy: 0.2670
Epoch 9/500
38561/38561 [==============================] - 18s 470us/step - loss: 4.3918 - accuracy: 0.2740
Epoch 10/500
38561/38561 [==

In [36]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
38561/38561 [==============================] - 18s 473us/step - loss: 0.7707 - accuracy: 0.8167
Epoch 2/100
38561/38561 [==============================] - 19s 483us/step - loss: 0.7710 - accuracy: 0.8171
Epoch 3/100
38561/38561 [==============================] - 18s 472us/step - loss: 0.7663 - accuracy: 0.8185
Epoch 4/100
38561/38561 [==============================] - 18s 473us/step - loss: 0.7632 - accuracy: 0.8187
Epoch 5/100
38561/38561 [==============================] - 18s 475us/step - loss: 0.7668 - accuracy: 0.8159
Epoch 6/100
38561/38561 [==============================] - 18s 473us/step - loss: 0.7582 - accuracy: 0.8209
Epoch 7/100
38561/38561 [==============================] - 18s 472us/step - loss: 0.7674 - accuracy: 0.8163
Epoch 8/100
38561/38561 [==============================] - 18s 473us/step - loss: 0.7588 - accuracy: 0.8203
Epoch 9/100
38561/38561 [==============================] - 18s 470us/step - loss: 0.7538 - accuracy: 0.8213
Epoch 10/100
38561/38561 [==

In [0]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [62]:
print(generate_seq(model, tokenizer, max_length-1, 'Mr. Bean ', 6))
print(generate_seq(model, tokenizer, max_length-1, 'A group of grave',8 ))
print(generate_seq(model, tokenizer, max_length-1, 'BEAN tries to cheer',12))

Mr. Bean  sees bernie in the airport scene
A group of grave gentlemen and gentlewomen they are the trustees of
BEAN tries to cheer him up he mimes an aeroplane which makes the boy feel n'


In [72]:
print(generate_seq(model, tokenizer, max_length-1, 'His eyes close, and',12))
print(generate_seq(model, tokenizer, max_length-1, 'His arm goes up',5))
print(generate_seq(model, tokenizer, max_length-1, 'BEAN then switches on',5))

His eyes close, and the framed white rear gallery keys to the first gallery walks up
His arm goes up and rings for the hostess
BEAN then switches on the noisy overhead air blower
